In [1]:

words = open('names.txt','r').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
len(words)

32033

In [4]:
min(len(w) for w in words)

2

In [5]:
max(len(w) for w in words)

15

In [6]:
b = {}
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        #print(ch1, ch2)

In [7]:
sorted(b.items(), key = lambda kv: -kv[1])[:10]

[(('n', '.'), 6763),
 (('a', '.'), 6640),
 (('a', 'n'), 5438),
 (('.', 'a'), 4410),
 (('e', '.'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('.', 'k'), 2963)]

In [8]:
import torch

In [9]:
N = torch.zeros((27,27), dtype=torch.int32)

In [10]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [11]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] += 1
        #print(ch1, ch2)

In [12]:
print(f"{itos[3]}, {stoi['c']}")

c, 3


In [13]:
N[:10,:10]

tensor([[   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591],
        [6640,  556,  541,  470, 1042,  692,  134,  168, 2332, 1650],
        [ 114,  321,   38,    1,   65,  655,    0,    0,   41,  217],
        [  97,  815,    0,   42,    1,  551,    0,    2,  664,  271],
        [ 516, 1303,    1,    3,  149, 1283,    5,   25,  118,  674],
        [3983,  679,  121,  153,  384, 1271,   82,  125,  152,  818],
        [  80,  242,    0,    0,    0,  123,   44,    1,    1,  160],
        [ 108,  330,    3,    0,   19,  334,    1,   25,  360,  190],
        [2409, 2244,    8,    2,   24,  674,    2,    2,    1,  729],
        [2489, 2445,  110,  509,  440, 1653,  101,  428,   95,   82]],
       dtype=torch.int32)

In [14]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [15]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [16]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
#ix
itos[ix]

'j'

In [17]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator = g)
p = p / p.sum()
p

tensor([0.6064, 0.3033, 0.0903])

In [18]:
torch.multinomial(p,num_samples=20,replacement=True,generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [24]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)
#P.sum(1, keepdim=True).shape
P[0].sum()

tensor(1.)

In [25]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix = 0
    while True:
        p = P[ix]
        #p = N[ix].float()
        #p = p/p.sum()
        #p = torch.ones(27)/27.0 # every character is equally likely
        ix = torch.multinomial(p,num_samples=1,replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.


In [26]:
log_likelihood = 0.0
n = 0
#for w in words[:3]:
for w in ['andrejq']:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1,ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

.a: 0.1376 -1.9835
an: 0.1604 -1.8302
nd: 0.0384 -3.2594
dr: 0.0770 -2.5646
re: 0.1334 -2.0143
ej: 0.0027 -5.9004
jq: 0.0003 -7.9817
q.: 0.0970 -2.3331
log_likelihood=tensor(-27.8672)
nll=tensor(27.8672)
nll/n=tensor(3.4834)


In [ ]:
# create the training set of all the biagrams

In [95]:
# create the training set of all the biagrams

xs, ys = [],[]
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
        n_elements += 1

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(f'{num=}')

num=228146


In [96]:
import torch.nn.functional as F
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True)

In [98]:
# gradient descent
for i in range(100):
    # Forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalient to N
    probs = counts/counts.sum(1, keepdim=True) # probabilities for next character
    loss = - probs[torch.arange(num),ys].log().mean() # negative, average log likelihood
    # Backward pass
    W.grad = None # Set to zero the gradient
    loss.backward()
    W.data += -50 * W.grad

print(loss.item())

2.462393045425415


In [102]:
# Inference - sampling from the 'neural net' model
for i in range(10):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalient to N
        p = counts/counts.sum(1, keepdim=True) # probabilities for next character
        ix = torch.multinomial(p,num_samples=1,replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

morvann.
akela.
az.
arileri.
chaiadayra.
fbalodouseyton.
za.
sabran.
han.
ke.
